# Análisis Cuantitativo para Tesis: Obras Públicas en Puno (2013-2025)

**Autor de la Tesis:** Bach. Fabricio Rigoberto Rodriguez Ascarrunz
**Análisis de Datos por:** Ing. Jhon Wilber Ajata Ascarrunz

**Objetivo:** Este notebook reproduce el análisis cuantitativo descrito en el Capítulo IV de la tesis. Se parte de la base de datos **certificada** de INFOBRAS para generar las tablas de resultados y realizar las pruebas de hipótesis que comparan las modalidades de ejecución por Administración Directa y por Contrata.

In [1]:
# --- Paso 1 y 2: Configuración del Entorno y Carga de Datos ---
import pandas as pd
import numpy as np
from scipy.stats import shapiro, mannwhitneyu

print("Librerías importadas correctamente.")

try:
    # CARGAMOS EL ARCHIVO CERTIFICADO, EL RESULTADO DE TODO NUESTRO TRABAJO ANTERIOR
    df_certificado = pd.read_csv('../data/infobras_certificado_v1.csv', low_memory=False)
    
    # Aseguramos que las columnas de fecha clave sean del tipo datetime
    date_cols_to_convert = ['fecha_de_inicio_de_obra', 'fecha_de_finalizacion_real']
    for col in date_cols_to_convert:
        df_certificado[col] = pd.to_datetime(df_certificado[col], errors='coerce')
    
    print("\nDataset 'infobras_certificado_v1.csv' cargado y preparado.")
    
    # Filtramos para obtener solo los registros de la región Puno
    df_puno = df_certificado[df_certificado['departamento'] == 'PUNO'].copy()
    print(f"Se han aislado {len(df_puno)} obras certificadas correspondientes a la región Puno.")

except FileNotFoundError:
    print("\nError Crítico: No se encontró el archivo 'infobras_certificado_v1.csv'.")
    print("Por favor, ejecuta primero el notebook '01_Data_Cleaning_INFOBRAS.ipynb' para generarlo.")

Librerías importadas correctamente.

Dataset 'infobras_certificado_v1.csv' cargado y preparado.
Se han aislado 8500 obras certificadas correspondientes a la región Puno.


In [2]:
# --- Paso 3: Definición y Refinamiento de la Muestra de Análisis (Sección 4.1) ---

# 1. Filtro inicial para obras finalizadas con datos financieros válidos y lógicos
initial_sample = df_puno[
    (df_puno['estado_de_ejecucion'] == 'Finalizado') &
    (df_puno['monto_viable/aprobado'] > 0) & 
    (df_puno['monto_de_ejecucion_financiera_de_la_obra'] > 0)
].copy()

print(f"Muestra inicial (obras finalizadas en Puno con datos válidos): {len(initial_sample)} registros.")

# 2. Refinamiento metodológico: Aislar solo los grupos de comparación de la tesis
modalidades_de_interes = ['Administracion Directa', 'Contrata']
analysis_sample = initial_sample[initial_sample['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print(f"Muestra refinada para comparación directa: {len(analysis_sample)} registros.")

# 3. Caracterización de la muestra final
print("\nComposición de la muestra final por modalidad:")
print(analysis_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

Muestra inicial (obras finalizadas en Puno con datos válidos): 401 registros.
Muestra refinada para comparación directa: 76 registros.

Composición de la muestra final por modalidad:
modalidad_de_ejecucion_de_la_obra
Contrata    76
Name: count, dtype: int64


In [3]:
# --- Paso 4: Análisis del Cumplimiento Presupuestal (Objetivo Específico 1) ---

# 1. Ingeniería de Característica: Desviación Presupuestal Porcentual (DP%)
analysis_sample['desviacion_presupuestal_%'] = \
    ((analysis_sample['monto_de_ejecucion_financiera_de_la_obra'] - analysis_sample['monto_viable/aprobado']) 
     / analysis_sample['monto_viable/aprobado']) * 100

print("\nIndicador 'desviacion_presupuestal_%' calculado.")

# 2. Separar los datos por modalidad
admin_directa_dp = analysis_sample[analysis_sample['modalidad_de_ejecucion_de_la_obra'] == 'Administracion Directa']['desviacion_presupuestal_%'].dropna()
contrata_dp = analysis_sample[analysis_sample['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']['desviacion_presupuestal_%'].dropna()

# 3. Generar la Tabla 2 de la tesis
tabla_2_dp = pd.DataFrame({
    'Administración Directa': admin_directa_dp.describe(),
    'Contrata': contrata_dp.describe()
}).T[['count', 'mean', '50%', 'std', 'min', 'max']]

tabla_2_dp.rename(columns={'50%': 'Mediana (DP%)', 'count': 'N', 'mean': 'Media (DP%)', 
                           'std': 'Desv. Estándar', 'min': 'Mínimo', 'max': 'Máximo'}, inplace=True)
tabla_2_dp['N'] = tabla_2_dp['N'].astype(int)

print("\n--- Tabla 2: Estadísticas Descriptivas de la Desviación Presupuestal Porcentual (DP%) ---")
display(tabla_2_dp.round(2))

# 4. Prueba de normalidad
print("\nPrueba de Normalidad (Shapiro-Wilk):")
print(f"  P-valor para Administración Directa: {shapiro(admin_directa_dp).pvalue:.4e}")
print(f"  P-valor para Contrata: {shapiro(contrata_dp).pvalue:.4e}")
print("--> Conclusión: Los datos no son normales (p < 0.05).")

# 5. Prueba de hipótesis
u_stat, p_value_dp = mannwhitneyu(admin_directa_dp, contrata_dp, alternative='two-sided')
print("\nPrueba de Hipótesis (U de Mann-Whitney) para Desviación Presupuestal:")
print(f"  Estadístico U = {u_stat:.0f}")
print(f"  p-value = {p_value_dp:.4f}")
if p_value_dp < 0.05:
    print("--> Conclusión: Existe una diferencia estadísticamente significativa (p < 0.05).")
else:
    print("--> Conclusión: No existe una diferencia estadísticamente significativa (p >= 0.05).")


Indicador 'desviacion_presupuestal_%' calculado.

--- Tabla 2: Estadísticas Descriptivas de la Desviación Presupuestal Porcentual (DP%) ---


,N,Media (DP%),Mediana (DP%),Desv. Estándar,Mínimo,Máximo
Administración Directa,0,NaN,NaN,NaN,NaN,NaN
Contrata,76,-2.94,-9.87,129.41,-100.0,1010.66



Prueba de Normalidad (Shapiro-Wilk):
  P-valor para Administración Directa: nan
  P-valor para Contrata: 5.3804e-16
--> Conclusión: Los datos no son normales (p < 0.05).

Prueba de Hipótesis (U de Mann-Whitney) para Desviación Presupuestal:
  Estadístico U = nan
  p-value = nan
--> Conclusión: No existe una diferencia estadísticamente significativa (p >= 0.05).


C:\Users\GAMERS\AppData\Local\Temp\ipykernel_23896\1515734781.py:29: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  print(f"  P-valor para Administración Directa: {shapiro(admin_directa_dp).pvalue:.4e}")
C:\Users\GAMERS\AppData\Local\Temp\ipykernel_23896\1515734781.py:34: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  u_stat, p_value_dp = mannwhitneyu(admin_directa_dp, contrata_dp, alternative='two-sided')


In [4]:
# --- Paso 3: Definición y Refinamiento de la Muestra de Análisis ---
print("\n--- Paso 3: Definición y Refinamiento de la Muestra de Análisis (Sección 4.1) ---")

# Filtro inicial para obras finalizadas con datos financieros válidos y lógicos
initial_sample = df_puno[
    (df_puno['estado_de_ejecucion'] == 'Finalizado') &
    (df_puno['monto_viable/aprobado'] > 0) & 
    (df_puno['monto_de_ejecucion_financiera_de_la_obra'] > 0)
].copy()

print(f"Muestra inicial (obras finalizadas en Puno con datos válidos): {len(initial_sample)} registros.")

# Refinamiento metodológico
modalidades_de_interes = ['Administracion Directa', 'Contrata']
analysis_sample = initial_sample[initial_sample['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print(f"\nMuestra refinada para comparación directa: {len(analysis_sample)} registros.")
print("\nComposición de la muestra final por modalidad:")
conteo_modalidad = analysis_sample['modalidad_de_ejecucion_de_la_obra'].value_counts()
print(conteo_modalidad)

# --- HALLAZGO CRÍTICO ---
if 'Administracion Directa' not in conteo_modalidad or conteo_modalidad['Administracion Directa'] == 0:
    print("\n\n*****************************************************************")
    print("HALLAZGO METODOLÓGICO FUNDAMENTAL:")
    print("No se encontraron registros de obras por 'Administración Directa' que cumplan")
    print("con todos los criterios para un análisis de desempeño financiero completo.")
    print("Esto impide una comparación estadística directa del cumplimiento presupuestal.")
    print("El análisis procederá de forma descriptiva para el grupo 'Contrata'.")
    print("*****************************************************************")


--- Paso 3: Definición y Refinamiento de la Muestra de Análisis (Sección 4.1) ---
Muestra inicial (obras finalizadas en Puno con datos válidos): 401 registros.

Muestra refinada para comparación directa: 76 registros.

Composición de la muestra final por modalidad:
modalidad_de_ejecucion_de_la_obra
Contrata    76
Name: count, dtype: int64


*****************************************************************
HALLAZGO METODOLÓGICO FUNDAMENTAL:
No se encontraron registros de obras por 'Administración Directa' que cumplan
con todos los criterios para un análisis de desempeño financiero completo.
Esto impide una comparación estadística directa del cumplimiento presupuestal.
El análisis procederá de forma descriptiva para el grupo 'Contrata'.
*****************************************************************


In [5]:
# --- Paso 4: Análisis del Cumplimiento Presupuestal ---
if len(analysis_sample) > 0 and 'Contrata' in analysis_sample['modalidad_de_ejecucion_de_la_obra'].unique():
    print("\n--- Análisis Descriptivo para la Modalidad por Contrata ---")
    
    # Ingeniería de Característica para Contrata
    analysis_sample['desviacion_presupuestal_%'] = \
        ((analysis_sample['monto_de_ejecucion_financiera_de_la_obra'] - analysis_sample['monto_viable/aprobado']) 
         / analysis_sample['monto_viable/aprobado']) * 100

    contrata_dp = analysis_sample[analysis_sample['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']['desviacion_presupuestal_%'].dropna()
    
    # Generar la tabla descriptiva solo para Contrata
    tabla_contrata = pd.DataFrame({'Contrata': contrata_dp.describe()}).T
    print("\nEstadísticas Descriptivas de Desviación Presupuestal para 'Contrata':")
    display(tabla_contrata)
else:
    print("\nNo hay datos suficientes para realizar el análisis presupuestal.")


--- Análisis Descriptivo para la Modalidad por Contrata ---

Estadísticas Descriptivas de Desviación Presupuestal para 'Contrata':


,count,mean,std,min,25%,50%,75%,max
Contrata,76.0,-2.93757,129.412936,-99.998591,-49.869381,-9.866222,3.564133,1010.662651


In [7]:
# --- Celda de Detective: Encontrando el nombre correcto de la columna de plazo ---

# Usamos el DataFrame df_puno que sabemos que está completo
todas_las_columnas = df_puno.columns.tolist()

# Buscamos columnas que contengan la palabra clave 'plazo'
columnas_de_plazo = [col for col in todas_las_columnas if 'plazo' in col]

print("Posibles nombres para la columna 'Plazo de Ejecución':")
print(columnas_de_plazo)

Posibles nombres para la columna 'Plazo de Ejecución':
['plazo_de_ejecucion_en_dias', 'n_dias_de_modificaciones_de_plazo', 'nuevo_plazo_de_ejecucion_en_dias']


In [8]:
# --- Paso 5: Análisis Comparativo de la Eficiencia en la Ejecución (Plazos) ---
print("--- Iniciando análisis de eficiencia en plazos (Objetivo Específico 2) ---")

# Tarea 2.1: Ingeniería de Característica - Variación del Plazo en días (VP días)
# Se usa el nombre de columna correcto: 'plazo_de_ejecucion_en_dias'
initial_sample['variacion_plazo_dias'] = \
    (initial_sample['fecha_de_finalizacion_real'] - initial_sample['fecha_de_inicio_de_obra']).dt.days - initial_sample['plazo_de_ejecucion_en_dias']

print("\nIndicador 'variacion_plazo_dias' calculado.")

# Tarea 2.2: Análisis de Calidad de Datos para el indicador de plazos
datos_completos_plazo = initial_sample['variacion_plazo_dias'].notna().sum()
total_muestra = len(initial_sample)
porcentaje_faltante = (1 - (datos_completos_plazo / total_muestra)) * 100

print(f"\nHallazgo sobre Calidad de Datos de Plazos:")
print(f"De {total_muestra} obras 'Finalizado' con datos financieros iniciales, solo {datos_completos_plazo} tienen datos consistentes para calcular la variación del plazo.")
print(f"Esto representa un {porcentaje_faltante:.1f}% de datos faltantes para este indicador.")

# Ahora creamos nuestra muestra específica para el análisis de plazos
plazo_analysis_sample = initial_sample.dropna(subset=['variacion_plazo_dias'])

print(f"\nLa muestra final para el análisis de plazos contiene {len(plazo_analysis_sample)} registros.")
print("\nComposición de la muestra de plazos por modalidad:")
print(plazo_analysis_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

--- Iniciando análisis de eficiencia en plazos (Objetivo Específico 2) ---

Indicador 'variacion_plazo_dias' calculado.

Hallazgo sobre Calidad de Datos de Plazos:
De 401 obras 'Finalizado' con datos financieros iniciales, solo 401 tienen datos consistentes para calcular la variación del plazo.
Esto representa un 0.0% de datos faltantes para este indicador.

La muestra final para el análisis de plazos contiene 401 registros.

Composición de la muestra de plazos por modalidad:
modalidad_de_ejecucion_de_la_obra
Administración directa    316
Contrata                   76
Por núcleo ejecutor         9
Name: count, dtype: int64


In [10]:
# --- Paso 5 y 6: Análisis Completo y Correcto de Eficiencia en Plazos (Sección 4.3) ---

print("--- Iniciando análisis de eficiencia en plazos con una muestra independiente ---")

# 1. CREAMOS UNA NUEVA MUESTRA PARA PLAZOS, partiendo de df_puno.
# El criterio es que la obra esté 'Finalizado' y tenga los datos de fechas y plazo programado.
plazo_sample = df_puno[
    (df_puno['estado_de_ejecucion'] == 'Finalizado') &
    (df_puno['fecha_de_inicio_de_obra'].notna()) &
    (df_puno['fecha_de_finalizacion_real'].notna()) &
    (df_puno['plazo_de_ejecucion_en_dias'].notna())
].copy()

# 2. Ingeniería de Característica: Variación del Plazo en días (VP días)
plazo_sample['variacion_plazo_dias'] = \
    (plazo_sample['fecha_de_finalizacion_real'] - plazo_sample['fecha_de_inicio_de_obra']).dt.days - plazo_sample['plazo_de_ejecucion_en_dias']

# 3. Refinamiento para la comparación directa
modalidades_de_interes = ['Administracion Directa', 'Contrata']
plazo_comparison_sample = plazo_sample[
    plazo_sample['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)
].copy()

print(f"\nMuestra final para la comparación de plazos: {len(plazo_comparison_sample)} registros.")
print("\nComposición de la muestra de plazos por modalidad:")
print(plazo_comparison_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

# 4. Separar los datos por modalidad
admin_directa_vp = plazo_comparison_sample[plazo_comparison_sample['modalidad_de_ejecucion_de_la_obra'] == 'Administracion Directa']['variacion_plazo_dias']
contrata_vp = plazo_comparison_sample[plazo_comparison_sample['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']['variacion_plazo_dias']

# 5. Generar la Tabla 3 de la tesis
tabla_3_vp = pd.DataFrame({
    'Administración Directa': admin_directa_vp.describe(),
    'Contrata': contrata_vp.describe()
}).T[['count', 'mean', '50%', 'std']]
tabla_3_vp.rename(columns={'50%': 'Mediana (días)', 'count': 'N', 'mean': 'Media (días)', 'std': 'Desv. Estándar'}, inplace=True)
tabla_3_vp['N'] = tabla_3_vp['N'].astype(int)

print("\n--- Tabla 3: Estadísticas Descriptivas de la Variación del Plazo (días) ---")
display(tabla_3_vp.round(0))

# 6. Realizar la prueba de hipótesis U de Mann-Whitney
u_stat_vp, p_value_vp = mannwhitneyu(admin_directa_vp, contrata_vp, alternative='two-sided')

print("\nPrueba de Hipótesis (U de Mann-Whitney) para Variación del Plazo:")
print(f"  Estadístico U = {u_stat_vp:.0f}")
print(f"  p-value = {p_value_vp:.4f}")
if p_value_vp < 0.05:
    print("--> Conclusión: Existe una diferencia estadísticamente significativa en la variación de plazos (p < 0.05).")
else:
    print("--> Conclusión: No existe una diferencia estadísticamente significativa en la variación de plazos (p >= 0.05).")

--- Iniciando análisis de eficiencia en plazos con una muestra independiente ---

Muestra final para la comparación de plazos: 1298 registros.

Composición de la muestra de plazos por modalidad:
modalidad_de_ejecucion_de_la_obra
Contrata    1298
Name: count, dtype: int64

--- Tabla 3: Estadísticas Descriptivas de la Variación del Plazo (días) ---


,N,Media (días),Mediana (días),Desv. Estándar
Administración Directa,0,NaN,NaN,NaN
Contrata,1298,153.0,30.0,2048.0



Prueba de Hipótesis (U de Mann-Whitney) para Variación del Plazo:
  Estadístico U = nan
  p-value = nan
--> Conclusión: No existe una diferencia estadísticamente significativa en la variación de plazos (p >= 0.05).


C:\Users\GAMERS\AppData\Local\Temp\ipykernel_23896\2575489620.py:44: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  u_stat_vp, p_value_vp = mannwhitneyu(admin_directa_vp, contrata_vp, alternative='two-sided')


In [13]:
# --- Paso 7: Análisis Final de Mecanismos de Control (Sección 4.4) ---
print("--- Iniciando análisis de mecanismos de control sobre TODA la población de Puno ---")

# Para este análisis, usamos el df_puno COMPLETO, sin filtrar por estado.
df_puno['fue_paralizado'] = (df_puno['causal_de_paralizacion'] != 'No Paralizada').astype(int)

# Refinamos para la comparación directa
modalidades_de_interes = ['Administracion Directa', 'Contrata']
control_comparison_sample = df_puno[
    df_puno['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)
].copy()

print(f"\nMuestra para la comparación de control: {len(control_comparison_sample)} registros.")
print("\nComposición de la muestra por modalidad:")
print(control_comparison_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

# Agrupamos por modalidad para calcular los indicadores
# SE USAN LOS NOMBRES DE COLUMNA CORRECTOS
control_analysis = control_comparison_sample.groupby('modalidad_de_ejecucion_de_la_obra').agg(
    total_obras=('codigo_infobras', 'count'),
    promedio_informes_control=('n_informes_de_control', 'mean'), # <-- CORREGIDO
    total_con_controversias=('n_de_controversias', lambda x: (x > 0).sum()),
    total_paralizadas=('fue_paralizado', 'sum')
)

# Calcular los porcentajes
control_analysis['Obras con Controversias (%)'] = (control_analysis['total_con_controversias'] / control_analysis['total_obras']) * 100
control_analysis['Tasa de Paralización (%)'] = (control_analysis['total_paralizadas'] / control_analysis['total_obras']) * 100

# Formatear la Tabla 4 de la tesis
tabla_4_control = control_analysis[['total_obras', 'promedio_informes_control', 'Obras con Controversias (%)', 'Tasa de Paralización (%)']]

print("\n--- Tabla 4: Indicadores de Transparencia y Control por Modalidad (Población Total) ---")
display(tabla_4_control.round(2))

--- Iniciando análisis de mecanismos de control sobre TODA la población de Puno ---

Muestra para la comparación de control: 1625 registros.

Composición de la muestra por modalidad:
modalidad_de_ejecucion_de_la_obra
Contrata    1625
Name: count, dtype: int64

--- Tabla 4: Indicadores de Transparencia y Control por Modalidad (Población Total) ---


,total_obras,promedio_informes_control,Obras con Controversias (%),Tasa de Paralización (%)
modalidad_de_ejecucion_de_la_obra,,,,
Contrata,1625,0.7,3.02,6.34


In [14]:
# --- Paso 7: Análisis Final de Mecanismos de Control (Sección 4.4) ---
print("--- Iniciando análisis de mecanismos de control sobre TODA la población de Puno ---")

# Para este análisis, usamos el df_puno COMPLETO.
df_puno['fue_paralizado'] = (df_puno['causal_de_paralizacion'] != 'No Paralizada').astype(int)

# Refinamos para la comparación directa CON LA CORRECCIÓN DE MAYÚSCULAS
modalidades_de_interes = ['Administracion directa', 'Contrata'] # <-- ¡AQUÍ ESTÁ LA CORRECCIÓN!
control_comparison_sample = df_puno[
    df_puno['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)
].copy()

print(f"\nMuestra para la comparación de control: {len(control_comparison_sample)} registros.")
print("\nComposición de la muestra por modalidad:")
print(control_comparison_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

# Agrupamos por modalidad para calcular los indicadores
control_analysis = control_comparison_sample.groupby('modalidad_de_ejecucion_de_la_obra').agg(
    total_obras=('codigo_infobras', 'count'),
    promedio_informes_control=('n_informes_de_control', 'mean'),
    total_con_controversias=('n_de_controversias', lambda x: (x > 0).sum()),
    total_paralizadas=('fue_paralizado', 'sum')
)

# Calcular los porcentajes
control_analysis['Obras con Controversias (%)'] = (control_analysis['total_con_controversias'] / control_analysis['total_obras']) * 100
control_analysis['Tasa de Paralización (%)'] = (control_analysis['total_paralizadas'] / control_analysis['total_obras']) * 100

# Formatear la Tabla 4 de la tesis
tabla_4_control = control_analysis[['total_obras', 'promedio_informes_control', 'Obras con Controversias (%)', 'Tasa de Paralización (%)']]

print("\n--- Tabla 4: Indicadores de Transparencia y Control por Modalidad (Población Total) ---")
display(tabla_4_control.round(2))

--- Iniciando análisis de mecanismos de control sobre TODA la población de Puno ---

Muestra para la comparación de control: 1625 registros.

Composición de la muestra por modalidad:
modalidad_de_ejecucion_de_la_obra
Contrata    1625
Name: count, dtype: int64

--- Tabla 4: Indicadores de Transparencia y Control por Modalidad (Población Total) ---


,total_obras,promedio_informes_control,Obras con Controversias (%),Tasa de Paralización (%)
modalidad_de_ejecucion_de_la_obra,,,,
Contrata,1625,0.7,3.02,6.34


In [15]:
import pandas as pd
import numpy as np

def clean_infobras_data(df_a_limpiar):
    """
    Esta función toma el DataFrame crudo de INFOBRAS y ejecuta una pipeline 
    completa de limpieza, auditoría y curación de datos. VERSIÓN TITANIO 4.0
    """
    df = df_a_limpiar.copy()
    print("Función de limpieza iniciada...")

    # --- PASITO 1: BLINDAJE Y NORMALIZACIÓN DE TODO EL TEXTO ---
    print("  -> Pasito 1: Blindando y normalizando todas las columnas de texto...")
    cols_texto = df.select_dtypes(include=['object']).columns
    for col in cols_texto:
        # Primero, eliminamos espacios al inicio y al final
        df[col] = df[col].str.strip()
        # SEGUNDO, NORMALIZAMOS PARA ELIMINAR ACENTOS Y CARACTERES ESPECIALES
        # Esto convierte 'Administración' en 'Administracion'
        df[col] = df[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    print(f"     - Se han blindado y normalizado {len(cols_texto)} columnas de texto.")

    # El resto de la función es la misma, pero ahora operará sobre datos 100% limpios
    print("  -> Pasito 2: Estandarizando nombres de columnas...")
    df.columns = (df.columns.str.lower().str.replace(' ', '_', regex=False).str.replace('¿', '', regex=False).str.replace('?', '', regex=False).str.replace('(', '', regex=False).str.replace(')', '', regex=False).str.replace(':', '', regex=False).str.replace('.', '', regex=False).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

    # ... [EL RESTO DE LOS PASOS 3, 4, 5, 5.5, 6, 7, y 8 SON EXACTAMENTE IGUALES] ...
    # (Pega aquí el resto de la función que ya tenías)
    print("  -> Pasito 3: Reparando estructura...")
    rename_dict = {
        'ruc1': 'ruc_supervisor', 'nombre_o_razon_social_de_la_empresa_o_consorcio1': 'razon_social_supervisor',
        'monto_del_contrato__en_soles1': 'monto_contrato_soles_supervisor', 'tipo_de_documento_de_identidad1': 'tipo_doc_identidad_residente',
        'numero_de_documento1': 'num_doc_residente', 'nombres_apellidos1': 'nombres_apellidos_residente',
        'colegiatura1': 'colegiatura_residente', 'numero_de_colegiatura1': 'num_colegiatura_residente',
        'fecha_inicio_de_labores': 'fecha_inicio_labores_residente', 'fecha_fin__de_labores': 'fecha_fin_labores_residente'
    }
    df = df.rename(columns=rename_dict)

    print("  -> Pasito 4: Podando columnas inútiles...")
    cols_a_podar = ['fecha_de_aprobacion', 'otra_marca', 'tipo_de_certificado_de_inversion_publica', 'numero_del_cipril_/cipgn', 'fecha_del_cipril_/_cipgn', 'monto_cipril_/cipgn']
    df = df.drop(columns=cols_a_podar, errors='ignore')
    
    print("  -> Pasito 5: Transformando columnas de fecha...")
    columnas_de_fecha = ['fecha_de_actualizacion', 'fecha_de_aprobacion_del_expediente', 'fecha_inicio_supervision', 'fecha_fin_supervision', 'fecha_inicio_labores_residente', 'fecha_fin_labores_residente', 'fecha_de_inicio_de_obra', 'fecha_finalizacion_programada_de_obra', 'fecha_de_entrega_del_terreno', 'fecha_de_registro_de_avance', 'fecha_de_paralizacion', 'fecha_finalizacion_reprogramada_de_obra', 'fecha_de_finalizacion_real', 'fecha_de_recepcion', 'fecha_de_aprobacion_de_liquidacion_de_obra', 'fecha_de_transferencia']
    for col in columnas_de_fecha:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')
            
    print("  -> Pasito 5.5: Auditando y corrigiendo coherencia de fechas...")
    mascara_inconsistente = (df['fecha_de_finalizacion_real'].notna()) & (df['fecha_de_inicio_de_obra'].notna()) & (df['fecha_de_finalizacion_real'] < df['fecha_de_inicio_de_obra'])
    inconsistencias_antes = mascara_inconsistente.sum()
    if inconsistencias_antes > 0:
        df = df[~mascara_inconsistente].copy()
    
    print("  -> Pasito 6: Decontaminando y transformando columnas numéricas...")
    columnas_numericas_contaminadas = ['monto_viable/aprobado', 'monto_de_aprobacion_de_expediente_tecnico', 'tasa_de_cambio', 'monto_aprobado_en_soles', 'monto_del_contrato__en_soles', 'monto_contrato_soles_supervisor', 'porcentaje_de_terreno_entregado', 'avance_fisico_programado_acumulado_%', 'avance_fisico_real_acumulado_%', 'monto_de_valorizacion_programado_acumulado', 'monto_de_valorizacion_ejecutado_acumulado', 'porcentaje_de_ejecucion_financiera', 'monto_de_ejecucion_financiera_de_la_obra', 'monto_de_adicionales_de_obra_en_soles', 'monto_de_adicionales_de_supervision_en_soles', 'monto_de_deductivos_de_obra_en_soles', 'costo_de_la_obra_en_soles', 'monto_total_devengado_del_proyecto']
    for col in columnas_numericas_contaminadas:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace('S/.', '', regex=False).str.replace(',', '', regex=False).str.replace('%', '', regex=False).str.strip()
            df[col] = pd.to_numeric(df[col], errors='coerce')

    print("  -> Pasito 7: Manejando valores nulos estratégicamente...")
    df['causal_de_paralizacion'] = df['causal_de_paralizacion'].fillna('No Paralizada')
    df['corresponde_a_un_saldo_de_obra'] = df['corresponde_a_un_saldo_de_obra'].fillna('No')
    columnas_categoricas_clave = ['departamento', 'provincia', 'distrito', 'modalidad_de_ejecucion_de_la_obra', 'estado_de_ejecucion']
    for col in columnas_categoricas_clave:
        if col in df.columns:
            df[col] = df[col].fillna('Desconocido')
    columnas_numericas = df.select_dtypes(include=np.number).columns.tolist()
    df[columnas_numericas] = df[columnas_numericas].fillna(0)
    columnas_object = df.select_dtypes(include=['object']).columns.tolist()
    for col in columnas_object:
        df[col] = df[col].fillna('No Aplica')
    
    print("  -> Pasito 8: Aplicando filtros de plausibilidad de negocio (Curación)...")
    df = df[df['plazo_de_ejecucion_en_dias'] >= 1]
    df = df[df['plazo_de_ejecucion_en_dias'] <= 3650]

    print("\n¡PROCESO DE LIMPIEZA, AUDITORÍA Y CURACIÓN COMPLETADO!")
    return df

In [16]:
# --- EJECUCIÓN FINAL Y EXPORTACIÓN DEL DATASET CERTIFICADO ---

# Cargar el dataset crudo original
df_raw = pd.read_excel('../data/DataSet-Obras-Publicas-23-07-2025.xlsx', engine='openpyxl')

# Ejecutar la NUEVA y DEFINITIVA pipeline de limpieza "Titanio"
df_certificado = clean_infobras_data(df_raw)

# Sobrescribir el archivo CSV con la versión certificada
df_certificado.to_csv('../data/infobras_certificado_v1.csv', index=False)

print("\n----------------------------------------------------")
print("✅  FASE 1 COMPLETADA")
print("Archivo 'infobras_certificado_v1.csv' ha sido actualizado con la versión 'Titanio'.")
print("Los datos ahora están completamente limpios, sin espacios ocultos ni acentos.")

Función de limpieza iniciada...
  -> Pasito 1: Blindando y normalizando todas las columnas de texto...
     - Se han blindado y normalizado 85 columnas de texto.
  -> Pasito 2: Estandarizando nombres de columnas...
  -> Pasito 3: Reparando estructura...
  -> Pasito 4: Podando columnas inútiles...
  -> Pasito 5: Transformando columnas de fecha...


C:\Users\GAMERS\AppData\Local\Temp\ipykernel_23896\3340891787.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')


  -> Pasito 5.5: Auditando y corrigiendo coherencia de fechas...
  -> Pasito 6: Decontaminando y transformando columnas numéricas...
  -> Pasito 7: Manejando valores nulos estratégicamente...
  -> Pasito 8: Aplicando filtros de plausibilidad de negocio (Curación)...

¡PROCESO DE LIMPIEZA, AUDITORÍA Y CURACIÓN COMPLETADO!

----------------------------------------------------
✅  FASE 1 COMPLETADA
Archivo 'infobras_certificado_v1.csv' ha sido actualizado con la versión 'Titanio'.
Los datos ahora están completamente limpios, sin espacios ocultos ni acentos.


In [17]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, mannwhitneyu

print("--- Cargando Dataset CERTIFICADO ---")
# Apuntamos a nuestro archivo de alta calidad
df_certificado = pd.read_csv('../data/infobras_certificado_v1.csv', low_memory=False)

# Aseguramos fechas
date_cols = ['fecha_de_inicio_de_obra', 'fecha_de_finalizacion_real']
for col in date_cols:
    df_certificado[col] = pd.to_datetime(df_certificado[col], errors='coerce')

# Filtramos para Puno. Ahora este filtro es 100% fiable.
df_puno = df_certificado[df_certificado['departamento'] == 'PUNO'].copy()
print(f"Se han aislado {len(df_puno)} obras certificadas de Puno.")
print("\nComposición por modalidad en Puno:")
print(df_puno['modalidad_de_ejecucion_de_la_obra'].value_counts())

--- Cargando Dataset CERTIFICADO ---
Se han aislado 8500 obras certificadas de Puno.

Composición por modalidad en Puno:
modalidad_de_ejecucion_de_la_obra
Administracion directa            6491
Contrata                          1625
Por nucleo ejecutor                373
Asociacion Publica Privada APP       6
Obras por impuestos                  3
Por convenio entre entidades         2
Name: count, dtype: int64


In [18]:
# --- ANÁLISIS PRESUPUESTAL ---
print("\n--- Preparando muestra para análisis presupuestal ---")
financiero_sample = df_puno[
    (df_puno['estado_de_ejecucion'] == 'Finalizado') &
    (df_puno['monto_viable/aprobado'] > 0) & 
    (df_puno['monto_de_ejecucion_financiera_de_la_obra'] > 0)
].copy()

# Usamos el nombre normalizado y sin acentos
modalidades_de_interes = ['Administracion directa', 'Contrata']
financiero_comparison_sample = financiero_sample[financiero_sample['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print("Composición de la muestra financiera:")
print(financiero_comparison_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

# Si hay datos en ambos grupos, procedemos
if not financiero_comparison_sample.empty and len(financiero_comparison_sample['modalidad_de_ejecucion_de_la_obra'].unique()) > 1:
    financiero_comparison_sample['desviacion_presupuestal_%'] = \
        ((financiero_comparison_sample['monto_de_ejecucion_financiera_de_la_obra'] - financiero_comparison_sample['monto_viable/aprobado']) 
         / financiero_comparison_sample['monto_viable/aprobado']) * 100

    admin_directa_dp = financiero_comparison_sample[financiero_comparison_sample['modalidad_de_ejecucion_de_la_obra'] == 'Administracion directa']['desviacion_presupuestal_%'].dropna()
    contrata_dp = financiero_comparison_sample[financiero_comparison_sample['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']['desviacion_presupuestal_%'].dropna()

    tabla_2_dp = pd.DataFrame({'Administracion directa': admin_directa_dp.describe(), 'Contrata': contrata_dp.describe()}).T
    print("\n--- Tabla 2: Estadísticas de Desviación Presupuestal (%) ---")
    display(tabla_2_dp)

    u_stat, p_value_dp = mannwhitneyu(admin_directa_dp, contrata_dp, alternative='two-sided')
    print(f"\nResultado Prueba U de Mann-Whitney: p-value = {p_value_dp:.4f}")
else:
    print("\nNo hay suficientes datos en ambas modalidades para una comparación presupuestal directa.")


--- Preparando muestra para análisis presupuestal ---
Composición de la muestra financiera:
modalidad_de_ejecucion_de_la_obra
Administracion directa    316
Contrata                   76
Name: count, dtype: int64

--- Tabla 2: Estadísticas de Desviación Presupuestal (%) ---


,count,mean,std,min,25%,50%,75%,max
Administracion directa,316.0,87.73370,1543.843466,-99.984988,-66.869545,-16.632180,1.673411,25436.008024
Contrata,76.0,-2.93757,129.412936,-99.998591,-49.869381,-9.866222,3.564133,1010.662651



Resultado Prueba U de Mann-Whitney: p-value = 0.2733


In [19]:
# --- ANÁLISIS DE CONTROL ---
print("\n--- Preparando muestra para análisis de control ---")
df_puno['fue_paralizado'] = (df_puno['causal_de_paralizacion'] != 'No Paralizada').astype(int)
control_comparison_sample = df_puno[df_puno['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print("Composición de la muestra de control:")
print(control_comparison_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

control_analysis = control_comparison_sample.groupby('modalidad_de_ejecucion_de_la_obra').agg(
    total_obras=('codigo_infobras', 'count'),
    promedio_informes_control=('n_informes_de_control', 'mean'),
    total_con_controversias=('n_de_controversias', lambda x: (x > 0).sum()),
    total_paralizadas=('fue_paralizado', 'sum')
)
control_analysis['Obras con Controversias (%)'] = (control_analysis['total_con_controversias'] / control_analysis['total_obras']) * 100
control_analysis['Tasa de Paralizacion (%)'] = (control_analysis['total_paralizadas'] / control_analysis['total_obras']) * 100
tabla_4_control = control_analysis[['total_obras', 'promedio_informes_control', 'Obras con Controversias (%)', 'Tasa de Paralizacion (%)']]

print("\n--- Tabla 4: Indicadores de Transparencia y Control ---")
display(tabla_4_control.round(2))


--- Preparando muestra para análisis de control ---
Composición de la muestra de control:
modalidad_de_ejecucion_de_la_obra
Administracion directa    6491
Contrata                  1625
Name: count, dtype: int64

--- Tabla 4: Indicadores de Transparencia y Control ---


,total_obras,promedio_informes_control,Obras con Controversias (%),Tasa de Paralizacion (%)
modalidad_de_ejecucion_de_la_obra,,,,
Administracion directa,6491,0.35,0.03,6.32
Contrata,1625,0.70,3.02,6.34


In [20]:
# --- ANÁLISIS DE PLAZOS (OBJETIVO ESPECÍFICO 2) ---
print("\n--- Preparando muestra para análisis de plazos ---")

# 1. CREAMOS UNA MUESTRA NUEVA E INDEPENDIENTE PARA PLAZOS
# El criterio es que la obra esté 'Finalizado' y tenga todos los datos de fechas y plazo necesarios.
plazo_sample = df_puno[
    (df_puno['estado_de_ejecucion'] == 'Finalizado') &
    (df_puno['fecha_de_inicio_de_obra'].notna()) &
    (df_puno['fecha_de_finalizacion_real'].notna()) &
    (df_puno['plazo_de_ejecucion_en_dias'].notna()) &
    (df_puno['plazo_de_ejecucion_en_dias'] > 0) # Añadimos el filtro de plausibilidad
].copy()

# 2. Refinamiento para la comparación directa
modalidades_de_interes = ['Administracion directa', 'Contrata']
plazo_comparison_sample = plazo_sample[
    plazo_sample['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)
].copy()

print(f"Muestra para la comparación de plazos: {len(plazo_comparison_sample)} registros.")
print("\nComposición de la muestra de plazos por modalidad:")
print(plazo_comparison_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

# Comprobamos si tenemos datos suficientes en ambos grupos para continuar
if not plazo_comparison_sample.empty and len(plazo_comparison_sample['modalidad_de_ejecucion_de_la_obra'].unique()) > 1:
    
    # 3. Ingeniería de Característica: Variación del Plazo en días
    plazo_comparison_sample['variacion_plazo_dias'] = \
        (plazo_comparison_sample['fecha_de_finalizacion_real'] - plazo_comparison_sample['fecha_de_inicio_de_obra']).dt.days - plazo_comparison_sample['plazo_de_ejecucion_en_dias']
        
    # 4. Separar los datos por modalidad
    admin_directa_vp = plazo_comparison_sample[plazo_comparison_sample['modalidad_de_ejecucion_de_la_obra'] == 'Administracion directa']['variacion_plazo_dias']
    contrata_vp = plazo_comparison_sample[plazo_comparison_sample['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']['variacion_plazo_dias']
    
    # 5. Generar la Tabla 3 de la tesis
    tabla_3_vp = pd.DataFrame({
        'Administracion directa': admin_directa_vp.describe(),
        'Contrata': contrata_vp.describe()
    }).T[['count', 'mean', '50%', 'std']]
    tabla_3_vp.rename(columns={'50%': 'Mediana (días)', 'count': 'N', 'mean': 'Media (días)', 'std': 'Desv. Estándar'}, inplace=True)
    tabla_3_vp['N'] = tabla_3_vp['N'].astype(int)

    print("\n--- Tabla 3: Estadísticas Descriptivas de la Variación del Plazo (días) ---")
    display(tabla_3_vp.round(0))
    
    # 6. Realizar la prueba de hipótesis U de Mann-Whitney
    u_stat_vp, p_value_vp = mannwhitneyu(admin_directa_vp, contrata_vp, alternative='two-sided')

    print("\nPrueba de Hipótesis (U de Mann-Whitney) para Variación del Plazo:")
    print(f"  Estadístico U = {u_stat_vp:.0f}")
    print(f"  p-value = {p_value_vp:.4f}")
    if p_value_vp < 0.05:
        print("--> Conclusión: Existe una diferencia estadísticamente significativa en la variación de plazos (p < 0.05).")
    else:
        print("--> Conclusión: No existe una diferencia estadísticamente significativa en la variación de plazos (p >= 0.05).")
else:
    print("\nHALLAZGO CRÍTICO: No se encontraron suficientes datos en ambas modalidades para una comparación de plazos directa.")
    print("Esto sugiere una brecha de información en el registro de fechas para al menos uno de los grupos.")


--- Preparando muestra para análisis de plazos ---
Muestra para la comparación de plazos: 5885 registros.

Composición de la muestra de plazos por modalidad:
modalidad_de_ejecucion_de_la_obra
Administracion directa    4587
Contrata                  1298
Name: count, dtype: int64

--- Tabla 3: Estadísticas Descriptivas de la Variación del Plazo (días) ---


,N,Media (días),Mediana (días),Desv. Estándar
Administracion directa,4587,118.0,27.0,308.0
Contrata,1298,153.0,30.0,2048.0



Prueba de Hipótesis (U de Mann-Whitney) para Variación del Plazo:
  Estadístico U = 2948618
  p-value = 0.5995
--> Conclusión: No existe una diferencia estadísticamente significativa en la variación de plazos (p >= 0.05).


In [21]:
# --- ANÁLISIS DE CONTROL (OBJETIVO ESPECÍFICO 3) ---
print("\n--- Preparando muestra para análisis de control (Población Total de Puno) ---")

# Para este análisis, usamos el df_puno COMPLETO
df_puno['fue_paralizado'] = (df_puno['causal_de_paralizacion'] != 'No Paralizada').astype(int)

# Refinamos para la comparación directa
modalidades_de_interes = ['Administracion directa', 'Contrata']
control_comparison_sample = df_puno[df_puno['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print("Composición de la muestra de control:")
print(control_comparison_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

# Agrupamos por modalidad para calcular los indicadores
control_analysis = control_comparison_sample.groupby('modalidad_de_ejecucion_de_la_obra').agg(
    total_obras=('codigo_infobras', 'count'),
    promedio_informes_control=('n_informes_de_control', 'mean'),
    total_con_controversias=('n_de_controversias', lambda x: (x > 0).sum()),
    total_paralizadas=('fue_paralizado', 'sum')
)

control_analysis['Obras con Controversias (%)'] = (control_analysis['total_con_controversias'] / control_analysis['total_obras']) * 100
control_analysis['Tasa de Paralizacion (%)'] = (control_analysis['total_paralizadas'] / control_analysis['total_obras']) * 100
tabla_4_control = control_analysis[['total_obras', 'promedio_informes_control', 'Obras con Controversias (%)', 'Tasa de Paralizacion (%)']]

print("\n--- Tabla 4: Indicadores de Transparencia y Control ---")
display(tabla_4_control.round(2))


--- Preparando muestra para análisis de control (Población Total de Puno) ---
Composición de la muestra de control:
modalidad_de_ejecucion_de_la_obra
Administracion directa    6491
Contrata                  1625
Name: count, dtype: int64

--- Tabla 4: Indicadores de Transparencia y Control ---


,total_obras,promedio_informes_control,Obras con Controversias (%),Tasa de Paralizacion (%)
modalidad_de_ejecucion_de_la_obra,,,,
Administracion directa,6491,0.35,0.03,6.32
Contrata,1625,0.70,3.02,6.34


In [22]:
# --- ANÁLISIS DE CONTROL (OBJETIVO ESPECÍFICO 3) ---
print("\n--- Preparando muestra para análisis de control (Población Total de Puno) ---")

# Para este análisis, usamos el df_puno COMPLETO
df_puno['fue_paralizado'] = (df_puno['causal_de_paralizacion'] != 'No Paralizada').astype(int)

# Refinamos para la comparación directa
modalidades_de_interes = ['Administracion directa', 'Contrata']
control_comparison_sample = df_puno[df_puno['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print("Composición de la muestra de control:")
print(control_comparison_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

# Agrupamos por modalidad para calcular los indicadores
control_analysis = control_comparison_sample.groupby('modalidad_de_ejecucion_de_la_obra').agg(
    total_obras=('codigo_infobras', 'count'),
    promedio_informes_control=('n_informes_de_control', 'mean'),
    total_con_controversias=('n_de_controversias', lambda x: (x > 0).sum()),
    total_paralizadas=('fue_paralizado', 'sum')
)

control_analysis['Obras con Controversias (%)'] = (control_analysis['total_con_controversias'] / control_analysis['total_obras']) * 100
control_analysis['Tasa de Paralizacion (%)'] = (control_analysis['total_paralizadas'] / control_analysis['total_obras']) * 100
tabla_4_control = control_analysis[['total_obras', 'promedio_informes_control', 'Obras con Controversias (%)', 'Tasa de Paralizacion (%)']]

print("\n--- Tabla 4: Indicadores de Transparencia y Control ---")
display(tabla_4_control.round(2))


--- Preparando muestra para análisis de control (Población Total de Puno) ---
Composición de la muestra de control:
modalidad_de_ejecucion_de_la_obra
Administracion directa    6491
Contrata                  1625
Name: count, dtype: int64

--- Tabla 4: Indicadores de Transparencia y Control ---


,total_obras,promedio_informes_control,Obras con Controversias (%),Tasa de Paralizacion (%)
modalidad_de_ejecucion_de_la_obra,,,,
Administracion directa,6491,0.35,0.03,6.32
Contrata,1625,0.70,3.02,6.34
